In [1]:
import numpy as np
import sympy as sym
import pydae.build as db
from pydae.grid_b_pu import bal_pu

In [2]:
file_path = 'test_hygov.json'
grid = bal_pu(file_path)

AVR added


In [3]:
params_dict = grid['params']

g_list = grid['g']
h_dict = grid['h']
f_list = grid['f']
x_list = grid['x']

sys = {'name':'test_hygov',
       'params_dict':params_dict,
       'f_list':f_list,
       'g_list':g_list,
       'x_list':x_list,
       'y_ini_list':grid['y'],
       'y_run_list':grid['y'],
       'u_run_dict':grid['u'],
       'u_ini_dict':grid['u'],
       'h_dict':h_dict}

sys = db.system(sys)
db.sys2num(sys)

In [4]:
g_list

[-P_B1/S_base + V_B1**2*g_B1_B2 + V_B1*V_B2*(-b_B1_B2*sin(theta_B1 - theta_B2) - g_B1_B2*cos(theta_B1 - theta_B2)) - S_n_B1*p_g_B1_1/S_base,
 -Q_B1/S_base + V_B1**2*(-b_B1_B2 - bs_B1_B2/2) + V_B1*V_B2*(b_B1_B2*cos(theta_B1 - theta_B2) - g_B1_B2*sin(theta_B1 - theta_B2)) - S_n_B1*q_g_B1_1/S_base,
 -P_B2/S_base + V_B1*V_B2*(b_B1_B2*sin(theta_B1 - theta_B2) - g_B1_B2*cos(theta_B1 - theta_B2)) + V_B2**2*g_B1_B2,
 -Q_B2/S_base + V_B1*V_B2*(b_B1_B2*cos(theta_B1 - theta_B2) + g_B1_B2*sin(theta_B1 - theta_B2)) + V_B2**2*(-b_B1_B2 - bs_B1_B2/2),
 R_a_B1*i_q_B1 + V_B1*cos(delta_B1 - theta_B1) + X1d_B1*i_d_B1 - e1q_B1,
 R_a_B1*i_d_B1 + V_B1*sin(delta_B1 - theta_B1) - X1q_B1*i_q_B1 - e1d_B1,
 V_B1*i_d_B1*sin(delta_B1 - theta_B1) + V_B1*i_q_B1*cos(delta_B1 - theta_B1) - p_g_B1_1,
 V_B1*i_d_B1*cos(delta_B1 - theta_B1) - V_B1*i_q_B1*sin(delta_B1 - theta_B1) - q_g_B1_1,
 K_a_B1*(-v_c_B1 + v_pss_B1 + v_ref_B1) + K_ai_B1*xi_v_B1 - v_f_B1,
 K_i_gov_B1*xi_omega_B1 + K_p_gov_B1*(-omega_B1 + omega_ref_B1) -

<img src=https://ars.els-cdn.com/content/image/1-s2.0-S1364032117308821-gr2_lrg.jpg>

In [8]:
params_dict = {'Droop':0.05,'A_t':1,'T_w':2.5,'D_t':1, 
               'K_p_gov':1.0, 'K_i_gov':0.1,
               'K_servo':1.0, 'T_servo':1.0,
               'V_gate_max':0.5, 'Gate_max':1.0,
                'H':5.0}  # parameters with default values


u_ini_dict = {'omega_ref':1.0, 'q_nl':0.0}  # input for the initialization problem
u_run_dict = {'omega_ref':1.0, 'q_nl':0.0}  # input for the running problem, its value is updated 


x_list = ['xi_omega','servo','pos','q','omega']  # dynamic states
y_ini_list = ['servo_u','gate','head','p_m']          # algebraic states for the initialization problem
y_run_list = ['servo_u','gate','head','p_m']        # algebraic for the running problem

sys_vars = {'params':params_dict,
            'u_list':u_run_dict,
            'x_list':x_list,
            'y_list':y_run_list}

exec(db.sym_gen_str())  # exec to generate the required symbolic varables and constants

<img src="htgsys2.png">
<img src="htgsys3.png">

In [ ]:
https://www.sciencedirect.com/science/article/pii/S1364032117308821

In [9]:
epsilon_omega = omega_ref - omega - Droop*gate
dxi_omega = epsilon_omega
g_servo_u = -servo_u + K_p_gov*epsilon_omega + K_i_gov*xi_omega
dservo = (K_servo*(servo_u-gate) - servo)/T_servo
dpos = sym.Piecewise((V_gate_max,servo>V_gate_max),(-V_gate_max,servo<-V_gate_max),(servo,True))
g_gate = -gate + sym.Piecewise((Gate_max,pos>Gate_max),(1e-6,pos<1e-6),(pos,True))
dq =  (1-head)/T_w
g_head = -head + (q/(gate+1e-6))**2 
g_p_m = -p_m + A_t*head*(q - q_nl)
domega = 1/(2*H)*(p_m - 0.1*omega)

In [ ]:
dx = 1/T*(u - x)
dx2 =1/T*(dx*A*T - x2)
